In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import numpy as np
import re
import pandas as pd
import sys

In [2]:
start_year = '2022'
end_year = '2022'

start_month = '08'
end_month = '08'

start_day = '08'
end_date = '08'

url = 'https://web.pcc.gov.tw/prkms/tender/common/agent/readTenderAgent?pageSize=3000&\
firstSearch=false&orgName=&orgId=&tenderName=&tenderId=&tenderStatus=TENDER_STATUS_1&\
tenderWay=TENDER_WAY_ALL_DECLARATION&awardAnnounceStartDate='+start_year+'%2F'+start_month+'%2F'+start_day+'&\
awardAnnounceEndDate='+end_year+'%2F'+end_month+'%2F'+end_date+'&radProctrgCate=RAD_PROCTRG_CATE_1&\
tenderRange=TENDER_RANGE_ALL&minBudget=&maxBudget=&item=&gottenVendorName=&\
gottenVendorId=&submitVendorName=&submitVendorId=&execLocation=&priorityCate=&radReConstruct='

url

'https://web.pcc.gov.tw/prkms/tender/common/agent/readTenderAgent?pageSize=3000&firstSearch=false&orgName=&orgId=&tenderName=&tenderId=&tenderStatus=TENDER_STATUS_1&tenderWay=TENDER_WAY_ALL_DECLARATION&awardAnnounceStartDate=2022%2F08%2F08&awardAnnounceEndDate=2022%2F08%2F08&radProctrgCate=RAD_PROCTRG_CATE_1&tenderRange=TENDER_RANGE_ALL&minBudget=&maxBudget=&item=&gottenVendorName=&gottenVendorId=&submitVendorName=&submitVendorId=&execLocation=&priorityCate=&radReConstruct='

In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
titles = soup.find_all('a', title='001')
keys = []

for title in titles:
    key = title.get('href')[-12:]
    keys.append(key)
    
len(keys)

147

In [4]:
datas = []

In [5]:
# print(soup.prettify())  #輸出排版後的HTML內容
# print(titles)
# print(key)
# keys

In [6]:
#中斷接續
# temp_keys = keys[533:]

In [7]:
site = 'https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain='

for key in tqdm(keys, position=0, leave=True):
    response = requests.get(site+key)
    soup = BeautifulSoup(response.text, "html.parser")

    #公告日
    result = soup.find('p').getText()
    date = result[5:]

    #標案資料
    columns = ['機關名稱', '標案案號', '標案名稱', '履約地點（含地區）', '廠商代碼', '廠商名稱', '廠商地址', '廠商電話', '決標金額']
    i = 0
    data = [site+key, date]
    titles = soup.find_all('tr', class_='tb_s06')

    for title in titles:
        text = title.getText().replace('\n', '').replace('\t', '').replace('\r', '').strip()
        res = [ele for ele in columns if(text.startswith(ele))]

        if (bool(res)):
            data.append(text)
            
        if text.startswith('決標金額'):
            break

    #     print(i, text)
        i+=1
        
    if data == [site+key, date]:
        break

    datas.append(data)
    time.sleep(15)

100%|██████████| 147/147 [39:26<00:00, 16.10s/it]


In [8]:
# print(soup.prettify())
# print(date)
# datas

In [9]:
temp_datas = datas

#刪除未得標廠商
for data in temp_datas:
    while len(data) > 11:
        data.pop(6)
        
temp_datas = np.reshape(temp_datas, (len(temp_datas), 11)).T

# 欄位整理
for i in range(2, 11):
    temp = []
    if i == 5:
        for j in temp_datas[i]:
            temp.append(j[9:])
        temp_datas[i] = temp
    elif i == 6:
        for j in temp_datas[i]:
            temp.append(j[4:].strip())
        temp_datas[i] = temp
    elif i == 10:
        for j in temp_datas[i]:
            temp.append(re.sub('\D', '', j))
        temp_datas[i] = temp
    else:
        for j in temp_datas[i]:
            temp.append(j[4:])
        temp_datas[i] = temp

temp = temp_datas[8].copy()
temp_datas[8] = temp_datas[10]
temp_datas[10] = temp_datas[9]
temp_datas[9] = temp

In [10]:
df = pd.DataFrame(temp_datas.T, columns=['標案網址', '公告日期', '機關名稱', '標案案號',
                                         '標案名稱', '履約地點（含地區）', '廠商代碼',
                                         '廠商名稱', '決標金額', '廠商地址', '廠商電話'])
try:
    df['決標金額'] = df['決標金額'].astype(int)
except:
    print(sys.exc_info()[0], "occurred.")
finally:
    df.to_excel('採購公報_'+start_year[2:]+start_month+start_day+'-'+end_year[2:]+end_month+end_date+'.xlsx')
df.head()

<class 'ValueError'> occurred.


,標案網址,公告日期,機關名稱,標案案號,標案名稱,履約地點（含地區）,廠商代碼,廠商名稱,決標金額,廠商地址,廠商電話
0,https://web.pcc.gov.tw/tps/atm/AtmAwardWithout...,111/08/09,臺灣港務股份有限公司高雄港務分公司,A0E220051-1,高雄港蓬萊港區基礎設施改善工程,高雄市－鼓山區,85778902,振勝營造有限公司,319827000,831高雄市大寮區上發六路1號2樓,(07)7875756
1,https://web.pcc.gov.tw/tps/atm/AtmAwardWithout...,111/08/08,高雄市桃源區樟山國民小學,ZAS11106,111年非山非市改善及充實教學環境工程-班級及專科教室(英語、音樂）,高雄市－桃源區,82800825,耘嵿設計顧問室內裝修有限公司,1843043,110臺北市信義區忠孝東路5段476號8樓,(02)2760057#203
2,https://web.pcc.gov.tw/tps/atm/AtmAwardWithout...,111/08/08,行政院農業委員會農業試驗所,1111072,訓練中心及作物大樓耐震補強工程,臺中市－霧峰區,72197657,王田營造股份有限公司,23570000,643雲林縣林內鄉中正路417號1樓,(05)5375660
3,https://web.pcc.gov.tw/tps/atm/AtmAwardWithout...,111/08/08,國立花蓮高級工業職業學校,111-09-02,原住民文化空間裝修工程,花蓮縣－花蓮,50746200,勝全土木包工業,979159,976花蓮縣光復鄉大華村中華路２０３號１樓,(0917)163073
4,https://web.pcc.gov.tw/tps/atm/AtmAwardWithout...,111/08/08,臺北市政府工務局,1110531C0104,111年度新孝悌樓教學空間優化及室內裝修工程,臺北市－內湖區,97289957,喜萊登室內裝修有限公司,16740000,110臺北市信義區莊敬路178巷2號2樓,(09)32035104


In [11]:
df.shape

(147, 11)